In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [7]:
a = pd.read_csv('final_data_train.csv')
b = a[:36000]
c = a[36000:]
b.to_csv('final_data_train_1.csv')
c.to_csv('final_data_train_2.csv')

In [8]:
train = pd.read_csv("final_data_train.csv")
test = pd.read_csv("final_data_test.csv")
y = train['dropout']
data = pd.concat([ train.drop(['dropout'], axis=1), test.drop(['dropout'], axis=1)], axis=0,ignore_index=True, sort=False) 

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72395 entries, 0 to 72394
Data columns (total 29 columns):
enrollment_id                     72395 non-null int64
dropout                           72395 non-null int64
username                          72395 non-null object
course_id                         72395 non-null object
from                              72395 non-null object
to                                72395 non-null object
nCourses                          72395 non-null int64
nEnrollments                      72395 non-null int64
avg_delay_chapters                22658 non-null float64
avg_delay_chapters_course         72395 non-null float64
lifetime                          72395 non-null int64
btw_course_gap_min                72395 non-null int64
btw_course_gap_max                72395 non-null int64
btw_course_gap_mean               72395 non-null float64
access                            72395 non-null int64
discussion                        72395 non-null int64
n

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96408 entries, 0 to 96407
Data columns (total 28 columns):
enrollment_id                     96408 non-null int64
username                          96408 non-null object
course_id                         96408 non-null object
from                              96408 non-null object
to                                96408 non-null object
nCourses                          96408 non-null int64
nEnrollments                      96408 non-null int64
avg_delay_chapters                30172 non-null float64
avg_delay_chapters_course         96408 non-null float64
lifetime                          96408 non-null int64
btw_course_gap_min                96408 non-null int64
btw_course_gap_max                96408 non-null int64
btw_course_gap_mean               96408 non-null float64
access                            96408 non-null int64
discussion                        96408 non-null int64
navigate                          96408 non-null int64
p

In [9]:
labels = [
 'nCourses',
 'nEnrollments',
 'lifetime_group',
 'access_group',
 'btw_course_gap_mean',
 'discussion',
 'navigate',
 'page_close',
 'problem',
 'video',
 'wiki',
 'problem_duration',
 'video_duration',
 'problem_duration_per_course',
 'video_duration_per_course']

In [10]:
data['access_group'] = pd.cut(data['access'], [-1,100,200,300,400,500,2200])

In [11]:
data['lifetime_group'] = pd.cut(data['lifetime'], [-1, 52, 110, 190,280 ])

In [12]:
data['md_interarrival_time'] = data['md_interarrival_time'].astype('int')

In [13]:
data['mean_interarrival_time'] = data['mean_interarrival_time'].astype('int')

In [14]:
data['problem_duration_per_course'] = data['problem_duration_per_course'].astype('int')

In [15]:
data['video_duration_per_course'] = data['video_duration_per_course'].astype('int')

In [16]:
data['discussion_duration_per_course'] = data['discussion_duration_per_course'].astype('int')

In [17]:
data['btw_course_gap_mean'] = data['btw_course_gap_mean'].astype('int')

In [19]:
T = data[labels]
D = pd.get_dummies(T)
TRAIN = D[:len(train)]
TEST = D[len(train):]

In [20]:
T.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96408 entries, 0 to 96407
Data columns (total 15 columns):
nCourses                       96408 non-null int64
nEnrollments                   96408 non-null int64
lifetime_group                 96408 non-null category
access_group                   96407 non-null category
btw_course_gap_mean            96408 non-null int64
discussion                     96408 non-null int64
navigate                       96408 non-null int64
page_close                     96408 non-null int64
problem                        96408 non-null int64
video                          96408 non-null int64
wiki                           96408 non-null int64
problem_duration               96408 non-null int64
video_duration                 96408 non-null int64
problem_duration_per_course    96408 non-null int64
video_duration_per_course      96408 non-null int64
dtypes: category(2), int64(13)
memory usage: 9.7 MB


In [21]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(TRAIN, y, test_size=0.2, random_state=0)

In [22]:
from sklearn.model_selection import cross_val_score as CVS

Models
================================================

In [34]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import RandomizedSearchCV as RSCV
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier

## Feature Selection

In [23]:
cat_features = list(range(0, TRAIN.shape[1]))

In [24]:


clf = CatBoostClassifier(
    iterations=650,
    learning_rate=0.25,
    random_seed=0
    #loss_function='CrossEntropy'
)



In [25]:
clf.fit(TRAIN, y, cat_features=cat_features, verbose=False)

In [33]:
intro = pd.DataFrame({'score' : clf.feature_importances_, 'features' : TRAIN.columns})
intro.sort_values(by='score')

,score,features
19,0.053437,"access_group_(200, 300]"
22,0.067973,"access_group_(500, 2200]"
21,0.072169,"access_group_(400, 500]"
20,0.143234,"access_group_(300, 400]"
18,0.327158,"access_group_(100, 200]"
14,0.413107,"lifetime_group_(52, 110]"
16,0.438082,"lifetime_group_(190, 280]"
13,0.449331,"lifetime_group_(-1, 52]"
15,0.622236,"lifetime_group_(110, 190]"
17,2.246912,"access_group_(-1, 100]"


In [35]:
model = SelectFromModel(clf, prefit=True)
TRAIN_R = model.transform(TRAIN)
print(TRAIN_R.shape)

TEST_R = model.transform(TEST)

(72395, 9)


## Trying different models

In [40]:
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.linear_model import LogisticRegression as LR
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier as RFC

In [41]:
gbc = GBC()
knn = KNN(n_neighbors=5)
lr = LR(solver='lbfgs', max_iter=1500)
svc = svm.SVC(gamma=0.1, C=1)
rfc = RFC(n_estimators=50,max_features='sqrt',  min_samples_split=6,min_samples_leaf=2, max_depth=20)

In [43]:
models = [gbc, knn, lr, svc, rfc, clf]

In [45]:
run = False
if run :
    for model in models:

        val = CVS(model, TRAIN_R, y, cv = 5, scoring='accuracy')
        print('Model : ' + str(model.__class__ )+' Score : '+ str(np.mean(val)))
# Model : <class 'sklearn.ensemble._gb.GradientBoostingClassifier'> Score : 0.8554320049727192
# Model : <class 'sklearn.neighbors._classification.KNeighborsClassifier'> Score : 0.8406519787278126
# Model : <class 'sklearn.linear_model._logistic.LogisticRegression'> Score : 0.848055804958906
# Model : <class 'sklearn.svm._classes.SVC'> Score : 0.7884246149595967
# Model : <class 'sklearn.ensemble._forest.RandomForestClassifier'> Score : 0.8554872574072796
# Model : <class 'catboost.core.CatBoostClassifier'> Score : 0.8524483735064576

In [46]:

params = {
    'max_depth' : [4, 6, 8, 20, 50],
    'n_estimators': [10, 30, 50, 60 ,],
    'max_features': ['sqrt', 'auto', 'log2'],
    'min_samples_split': [2, 3, 10],
    'min_samples_leaf': [1, 3, 10],
    'bootstrap': [True, False],
    }
rfc2 = RFC()
cross_validation = StratifiedKFold(n_splits=5)

search = RSCV(rfc2, scoring='accuracy', param_distributions = params,cv=cross_validation)

search.fit(TRAIN_R, y)

print('Best score: {}'.format(search.best_score_))

print('Best parameters: {}'.format(search.best_params_))
# Best score: 0.855970716209683
# Best parameters: {'n_estimators': 60, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': True}

Best score: 0.855970716209683
Best parameters: {'n_estimators': 60, 'min_samples_split': 2, 'min_samples_leaf': 10, 'max_features': 'auto', 'max_depth': 50, 'bootstrap': True}


In [52]:
search.cv_results_['mean_test_score'].min()

0.8485668899785896

In [53]:
search.cv_results_['mean_test_score'].max()

0.855970716209683

There is not much difference between min and max of test score, which suggest model may not be overfitting

In [54]:
rfc2 = RFC(n_estimators = 60, min_samples_split = 2, min_samples_leaf =  10, max_features = 'auto', max_depth = 50, bootstrap = True)

In [55]:
rfc2.fit(TRAIN, y)

RandomForestClassifier(max_depth=50, min_samples_leaf=10, n_estimators=60)

In [56]:
yp = rfc2.predict(TEST)

In [57]:
Accuracy = (yp == test['dropout']).sum()/len(test)
# 0.8587015366676384

0.8587015366676384